In [4]:
import pandas as pd
import os
import shutil
import subprocess

In [5]:
import pandas as pd

file_path = 'PN16-1_data.txt'

# Carregar a primeira linha como cabeçalho
df_header = pd.read_csv(file_path, nrows=1)

# Definir o intervalo de linhas a serem lidas
start_row = 0  # Linha inicial (após o cabeçalho, 0-indexed)
end_row = 3741182 + 200000    # Linha final (linha a ser lida, inclusive)

# Carregar o intervalo específico de linhas
df = pd.read_csv(file_path, skiprows=range(1, start_row), nrows=end_row - start_row + 1, header=None)

# Definir as colunas do DataFrame com base no cabeçalho
df.columns = df_header.columns

print(df)

C:\Users\orlando\AppData\Local\Temp\ipykernel_44424\648806710.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=range(1, start_row), nrows=end_row - start_row + 1, header=None)


                Time       1       2       3       4       5      6       7  \
0               Time   1.000   2.000   3.000   4.000   5.000  6.000   7.000   
1           0.000000   4.250   1.750  -1.125 -15.500 -35.750  6.500 -30.500   
2           0.001953   2.625   0.750  -1.500 -13.750 -33.250  4.375 -27.875   
3           0.003906  -1.000  -1.625  -1.125  -8.125 -17.875 -1.000 -21.125   
4           0.005859   0.625   0.125   0.250  -4.500  -8.375  2.375 -13.750   
...              ...     ...     ...     ...     ...     ...    ...     ...   
3941178  7697.611328 -80.125  18.500  64.000  36.875  54.625  3.125  72.750   
3941179  7697.613281 -71.625  21.875  64.000  38.375  57.625  3.875  82.250   
3941180  7697.615234 -71.000  22.500  61.875  39.125  61.875  4.125  83.875   
3941181  7697.617188 -72.125  19.375  61.250  36.750  59.500  1.750  75.000   
3941182  7697.619141 -79.375  10.250  61.250  34.250  55.250  0.500  72.750   

              8       9  ...      40       41      

General names
- 16_Fp1_General  BLACK
- 16_O1_General   RED
- 16_Cz_General   GREEN
- 16_Fp2_General  BLUE
- 16_O2_General   Cyan

Pre-ictal names
- 16_Fp1_Pre-ictal  BLACK
- 16_O1_Pre-ictal   RED
- 16_Cz_Pre-ictal   GREEN
- 16_Fp2_Pre-ictal  BLUE
- 16_O2_Pre-ictal   Cyan

Ictal names
- 16_Fp1_Ictal      BLACK
- 16_O1_Ictal       RED
- 16_Cz_Ictal       GREEN
- 16_Fp2_Ictal      BLUE
- 16_O2_Ictal       Cyan

Post-ictal names
- 16_Fp1_Post-ictal  BLACK
- 16_O1_Post-ictal   RED
- 16_Cz_Post-ictal   GREEN
- 16_Fp2_Post-ictal  BLUE
- 16_O2_Post-ictal   Cyan

In [6]:
channels_columns = [
    ['1', 'C1'],   # B -> channel 1,  Fp1 Black
    ['5', 'C5'],   # F -> channel 5,  O1 Red
    ['15', 'C15'], # P -> channel 15, Cz Green
    ['17', 'C17'], # R -> channel 17, Fp2 Blue
    ['21', 'C21']  # V -> channel 21  O2 Cyan
]

In [7]:
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split('.'))
    return h * 3600 + m * 60 + s

In [8]:
# Seizure n 1 (in sleep):
# File name: PN16-1.edf 
# Registration start time: 20.45.21
# Registration end time: 23.03.11
# Seizure start time: 22.45.05 
# Seizure end time: 22.47.08

registration_start_time_sec = time_to_seconds('20.45.21')
seizure_start_time_sec = time_to_seconds('22.45.05')
seizure_end_time_sec = time_to_seconds('22.47.08')

start_line_ictal = (seizure_start_time_sec - registration_start_time_sec) * 512 - 2 # Taking the first two lines out of the count
end_line_ictal =   (seizure_end_time_sec - registration_start_time_sec) * 512   - 2 # Taking the first two lines out of the count

# Show the results
print(f"Start Line Ictal: {start_line_ictal}")
print(f"End Line Ictal: {end_line_ictal}")

Start Line Ictal: 3678206
End Line Ictal: 3741182


## Cutting the ICTAL series

In [9]:
result_folder = 'ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_ictal:end_line_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the GENERAL series

In [10]:
start_line_general = (start_line_ictal + end_line_ictal)/2 - 76800 # Calculating the general starting point based on 0.2.30 (2 minutes and 30 seconds) before the ictal starts
end_line_general = (start_line_ictal + end_line_ictal)/2 + 76800   # Calculating the general ending point based on 0.2.30 (2 minutes and 30 seconds) after the ictal ends

result_folder = 'GENERAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_general:end_line_general, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the PRE ICTAL series

In [11]:
start_line_pre_ictal = start_line_ictal - (abs(start_line_ictal - end_line_ictal))  # Calculating the start of the pre-ictal based on the start of the ictal
end_line_pre_ictal = start_line_ictal 

result_folder = 'PRE_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_pre_ictal:end_line_pre_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the POST ICTAL series

In [12]:
start_line_post_ictal = end_line_ictal 
end_line_post_ictal = end_line_ictal + (abs(start_line_ictal - end_line_ictal)) # Calculating the end of the post-ictal based on the end of the ictal

result_folder = 'POST_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_post_ictal:end_line_post_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

# Running DFA

In [13]:
def run_dfa_exe(directory, input_files, output_files):
    dfa_exe_path = 'dfa12.exe'
    
    command = [dfa_exe_path]
    
    input_str = (
        f"{input_files[0]}\n"
        f"{input_files[1]}\n"
        f"{output_files[0]}\n"
        f"{output_files[1]}\n"
    )
    
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=input_str)
    
    if process.returncode != 0:
        print(f"Erro ao executar dfa12.exe na pasta {directory}: {stderr}")
    else:
        print(f"Saída do dfa12.exe na pasta {directory}:\n{stdout}")

def process_directory(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.txt')]
    files.sort() 

    for i in range(0, len(files), 2):
        if i + 1 < len(files):
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i + 1])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat')), os.path.join(directory, files[i + 1].replace('.txt', '.dat'))]

            print("input_files a:", input_files)
            print("output_files a:", output_files)
            run_dfa_exe(directory, input_files, output_files)
        else:
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat'))] * 2

            print("input_files b:", input_files)
            print("output_files b:", output_files)
            run_dfa_exe(directory, input_files, output_files)

directories = ['GENERAL', 'ICTAL', 'PRE_ICTAL', 'POST_ICTAL']

for directory in directories:
    if os.path.exists(directory):
        process_directory(directory)
    else:
        print(f"Pasta {directory} não encontrada.")


input_files a: ['GENERAL\\C1.txt', 'GENERAL\\C15.txt']
output_files a: ['GENERAL\\C1.dat', 'GENERAL\\C15.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da segunda serie a ser lida:
(....com o mesmo numero de pontos N):
Qual o nome do primeiro arquivo de saida (para o DFA1)?
Qual o nome do segundo arquivo de saida (para o DFA2)?

input_files a: ['GENERAL\\C17.txt', 'GENERAL\\C21.txt']
output_files a: ['GENERAL\\C17.dat', 'GENERAL\\C21.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da segunda serie a ser lida:
(....com o mesmo numero de pontos N):
Qual o nome do primeiro arquivo de saida (para o DFA1)?
Qual o nome do segundo arquivo de saida (para o DFA2)?

input_files b: ['GENERAL\\C5.txt', 'GENERAL\\C5.txt']
output_files b: ['GENERAL\\C5.dat', 'GENERAL\\C5.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da seg